In [1]:
# For NLP
import nltk
# natural language toolkit for human response understanding
nltk.download('punkt') #for tokenization of sentences into words
from nltk.stem.lancaster import LancasterStemmer  #used in reducing the different form of words into single base word
#like cook, cooked, cooking into base/stem word: cook
stemmer=LancasterStemmer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
pip install tflearn

  Using cached tflearn-0.5.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Tensorflow processing
import tensorflow as tf
import numpy as np  
import tflearn #for different layers in deep neural netwrok
import random #to generate the numbers
import json #to read the intent json file and train the model

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [7]:
from google.colab import files
files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# importing the chatbot intents
with open('intents.json') as json_data:
  intents=json.load(json_data)

In [ ]:
intents

In [ ]:
words=[] #storing individual words
classes=[] #storing the tags/classes of intent
documents=[] 
ignore=['?','!'] #punctuation/speacial marks to be excluded from words

for intent in intents['intents']:
  for pattern in intent['patterns']:
    # tokenization of each word in every sentence/pattern of patterns
    w=nltk.word_tokenize(pattern)
    words.extend(w) #adding individual words in empty list 
    documents.append((w,intent['tag'])) #adding words in documents for each tags
    # adding each tags in classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [ ]:
# converting into lowercase and removing duplicates and stemming of words
words=[stemmer.stem(w.lower()) for w in words if w not in ignore] #converts in lowercase+stemming+ignoring the ignore words
words=sorted(list(set(words))) #removing the duplicate words by set operation & storing alphabetically

In [ ]:
# removing duplicate classes
classes=sorted(list(set(classes)))

print(len(documents),"documents")
print(len(classes),"classses",classes)
print(len(words),"unique stemmed words",words)

In [ ]:
# creating the training data set
training=[]
output=[]
# create empty array for output
output_empty=[0]*len(classes)

# creating training set, bag of words for each sentence
for doc in documents:
  # for bag of words array
  bag=[]
  # list of tokenized words
  pattern_words=doc[0]
  # stemming each word
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
  # for current tag append 1 and for rest append 0 below 
  # by matching the patternword with words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  # for current tag output is 1 and 0 for rest other tags
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1

  training.append([bag,output_row])

# shuffling features and turning into np.array
random.shuffle(training)
training=np.array(training)

# creating training lists
# features(intent patterns)
trainx=list(training[:,0])
# labels(associated tags or labelled class)
trainy=list(training[:,1])

In [ ]:
# restting underlying graph data
tf.compat.v1.reset_default_graph()

# building the neural network
# input layer
net=tflearn.input_data(shape=[None,len(trainx[0])])
# 2 hidden layers
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
# ouptut layer having trainy[0] number of fetaures
net=tflearn.fully_connected(net,len(trainy[0]),activation="softmax")
# applying the regression on the input
net=tflearn.regression(net)


# dnn model
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

# start training
# more number of epochs more accuarcy, show_metric for showing the performancereprot
model.fit(trainx,trainy,n_epoch=1000,batch_size=8,show_metric=True)
# saving the model so that no need to train again and again
# but if new intents are provided then we have to retrain the model
model.save('model.tflearn')



In [ ]:
import pickle
#saving the model and intermediate data structrue
pickle.dump({'words':words,'classes':classes,'trainx':trainx,'trainy':trainy},open("training_data","wb"))

In [ ]:
pickle.dump(model,open("trainedmodel.pkl",'wb'))

In [ ]:
# restoring the all data structrues
data=pickle.load(open('training_data','rb'))
words=data['words']
classes=data['classes']
trainx=data['trainx']
trainy=data['trainy']

In [ ]:
with open('intents.json') as json_data:
  intents=json.load(json_data)

In [ ]:
# loading the saved model
model.load('./model.tflearn')

In [ ]:
# we will take inputs from user
# preprocess it into bag of words
def clean_sentences(sentence):
  # tokening the pattern
  sentence_words=nltk.word_tokenize(sentence)
  # stemming each word
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words


# returning the bag of words array as 0/1 for each word in bag
def bow(sentence,words,show_details=False):
  # tokeninz the pattern
  sentence_words=clean_sentences(sentence)
  # generating bag of words
  bag=[0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
        if show_details:
          print("found in abd: %s",w)
  return (np.array(bag))


**Response** **Processor**

In [ ]:
# WITHOUT CONTEXT

Error_threshold=0.30
def classify(sentence):
  # generate probabilities from model
  results=model.predict([bow(sentence,words)])[0]
  # filter out rpedictions below a threshold
  results=[[i,r] for i,r in enumerate(results) if r>Error_threshold]
  # sorting the strength of probabilties
  results.sort(key=lambda x:x[1],reverse=True)
  returnlist=[]
  for r in results:
    returnlist.append((classes[r[0]],r[1]))
  # return tuple of intent and probability 
  return returnlist


# here userID is the context about which conversations
def response(sentence, userID='123',show_details=False):
  results=classify(sentence)
  # if we hahve a classification then find the best mathcng intent tag
  if results:
    # loop as long as there are matches to process
    while results:
      for i in intents['intents']:
        # find  a tag matching the first result
        if i['tag']==results[0][0]:
          # a random respose from the intent
          return print(random.choice(i['responses']))
      results.pop(0)

In [5]:
# lets ask questions from chatbot
classify('Good Morning')

NameError: name 'classify' is not defined

In [6]:
response('Bye')

NameError: name 'response' is not defined

In [ ]:
# # WITH CONTEXT

# context={}
# Error_threshold=0.30
# def classify(sentence):
#   # generate probabilities from model
#   results=model.predict([bow(sentence,words)])[0]
#   # filter out rpedictions below a threshold
#   results=[[i,r] for i,r in enumerate(results) if r>Error_threshold]
#   # sorting the strength of probabilties
#   results.sort(key=lambda x:x[1],reverse=True)
#   returnlist=[]
#   for r in results:
#     returnlist.append((classes[r[0]],r[1]))
#   # return tuple of intent and probability 
#   return returnlist


# # here userID is the context about which conversations
# def response(sentence, userID='123',show_details=False):
#   results=classify(sentence)
#   # if we hahve a classification then find the best mathcng intent tag
#   if results:
#     # loop as long as there are matches to process
#     while results:
#       for i in intents['intents']:
#         # find  a tag matching the first result
#         if i['tag']==results[0][0]:
#           #set context for this intent if necessary
#           if 'context_set' in i:
#             if show_details:print('context:',i['context_set'])
#             context[userID]=i['context_set']
          
#           # check if this intent is contextual and applies to users conversation
#           if not 'context_filter' in i or \
#           (userID in context and 'context_filter' in i and i['context_filter']==context):
#             if show_details:print('tag:',i['tag'])
#             # a random respose from the intent
#             return print(random.choice(i['responses']))
#       results.pop(0)


In [ ]:
# response("bye")